# Macrostate Calculation Troubleshooting
This notebook looks at modifying code from David Mobley that has been used to compute macrostate pKa values using microstate predictions.

In [1]:
import os
import glob
import io
import collections
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats
from scipy.special import logsumexp
from scipy.optimize import fsolve
from pKa_macrostate_analysis import pKaSubmission
from pKa_macrostate_analysis import Macro_pKa

## Load Data
Load data from pickle file generated from *macrostate_analysis.ipynb*

In [2]:
microstate_dt_file = open("microstates.pickle","rb")
microstate_dt = pickle.load(microstate_dt_file)
microstate_dt_file.close()

## Macro pKa Calculation Function
First loads free energy and population charge functions before loading actual *get_macropKa* function in a separate cell.

In [3]:
# Compute beta and other constants
kB = 1.381 * 6.02214 / 1000.0  # [kJ/(mol K)]
beta = 1. / (kB * 300)  # [mol/kJ]
beta = beta * 4.186
C_unit = 1 / beta * np.log(10)


# Compute free energy as a function of pH for states
# WITHIN-charge transitions have same pH dependence
def DeltaG(pH, state, state_details):
    #print("state_details",state_details)
    for item in state_details:
        if item[0] == state:
            # 0 serves as the reference state; all transitions are away from 0.
            if item[2] == -1:
                DeltaM = 1
            elif item[2] == 1:
                DeltaM = -1
            elif item[2] ==2:
                DeltaM=-2
            else:
                DeltaM = 0  # Hack to capture fact that pH dependence of states at same formal charge is same/cancels
            # Compute value
            return (item[1] - pH * DeltaM * C_unit)  # Gunner eq 3


# Compute populations for charge states (without normalization, due to laziness/since it'll drop out)
def pop_charge(pH, formal_charge, state_details):
    free_energies = []
    for item in state_details: #state_details [('SM42_micro001', 0.5304000000000001, -1, 0.0, 1.3872000000000002)
        if item[2] == formal_charge:
            free_energies.append(-beta * DeltaG(pH, item[0], state_details))
    if formal_charge == 0:
        free_energies.append(0 * pH)
    #print("free_energies",free_energies)
    return np.exp(logsumexp(free_energies))

# get G of each group
def getG(msgroup):
    Pi_raw = np.array([np.exp(-beta*ms[1]) for ms in msgroup])
    Pi_norm = Pi_raw/sum(Pi_raw)
    E = sum(np.array([ms[1] for ms in msgroup]) * Pi_norm)
    TS = -sum(Pi_norm * np.log(Pi_norm))/beta
    G = E - TS
    return G

In [4]:
def get_macropka(rfe_data):
    macropkas = []
    # Extract each molecule
    molecules = {}
    for index, row in rfe_data.iterrows():
        SM = index
        state = row["ID tag"]
        charge = row["total charge"]
        rfe = row["pKa mean"]
        sem = row["pKa SEM"]
        model_uncertainty = row["pKa model uncertainty"]

        if SM in molecules:
            molecules[SM].append((state, rfe, charge, sem, model_uncertainty))
        else:
            molecules[SM] = [(state, rfe, charge, sem, model_uncertainty)]



    # Loop over molecules, convert to state_details
    SM_names = [x for x in molecules.keys()]
    print(SM_names)
    SM_names.sort()
    for sm_name in SM_names:
        state_details = molecules[sm_name]

        # Figure out what formal charges are present in states
        formal_charges = [info[2] for info in state_details]
        print(formal_charges)

        # group microstates into groups based their formal charge
        msgroup_p2 = [state for state in state_details if state[2] == 2]  # microstates with formal charge +2
        msgroup_p1 = [state for state in state_details if state[2] == 1]  # microstates with formal charge +1
        msgroup_p0 = [state for state in state_details if state[2] == 0]  # microstates with formal charge 0
        msgroup_p0.append(("reference state", 0, 0))  # add back reference state
        msgroup_n1 = [state for state in state_details if state[2] == -1]  # microstates with formal charge -1

        # for reaction A -> B
        # ΔGAB = (-1)(C_unit)(pH - pKaBA)
        # Therefore when pH = 0, we have pKaBA = ΔGAB/C_unit
        # TODO: How are these formal charges estimated?
        # Compute +2 to +1 transition
        if 2 in formal_charges:
            pka = Macro_pKa()
            pka.molecule = sm_name.split("_")[0]
            pka.transition_from = 2
            pka.transition_to = 1
            pka.SEM = state_details[0][3]
            pka.MU = state_details[0][4]

            # titration method given my David's group
            init_guess = -15
            func_2to1 = lambda pH : (pop_charge(pH, 2, state_details) - pop_charge(pH, 1, state_details))
            pH_solution_2to1, infodict, ier, mesg = fsolve(func_2to1, init_guess, factor = 0.1, full_output=True)
            # If message indicates poor convergence, change initial guess and try again
            if 'The iteration is not making good progress' in mesg:
                init_guess-=5
            pH_solution_2to1, infodict, ier, mesg = fsolve(func_2to1, init_guess, factor = 0.1, full_output=True)
            # If still poor convergence, print warning (MAY NEED BETTER SOLUTION TO THIS)
            if 'The iteration is not making good progress' in mesg:
                print(sm_name)
                print("WARNING: Numerical problems encountered with fsolv")
            pka.pKa_bytitration = pH_solution_2to1

            # delta G method given by Junjun Mao
            dG = getG(msgroup_p1) - getG(msgroup_p2)
            pka.pKa_bydG = (dG / C_unit)

            macropkas.append(pka)

        # Compute +1 to 0 transition
        if 1 in formal_charges:
            pka = Macro_pKa()
            pka.molecule = sm_name.split("_")[0]
            pka.transition_from = 1
            pka.transition_to = 0
            pka.SEM = state_details[0][3]
            pka.MU = state_details[0][4]

            # titration method given by David's group
            init_guess = -5
            func_10 = lambda pH: (pop_charge(pH, 1, state_details) - pop_charge(pH, 0, state_details))
            pH_solution_1to0, infodict, ier, mesg = fsolve(func_10, init_guess, factor=0.1, full_output=True)
            if 'The iteration is not making good progress' in mesg:
                init_guess-=3
            pH_solution_1to0, infodict, ier, mesg = fsolve(func_10, init_guess, factor=0.1, full_output=True)
            # If still poor convergence, print warning (MAY NEED BETTER SOLUTION TO THIS)
            if 'The iteration is not making good progress' in mesg:
                print(sm_name)
                print("WARNING: Numerical problems encountered with fsolv")
            pka.pKa_bytitration = pH_solution_1to0

            # delta G method given by Junjun Mao
            dG = getG(msgroup_p0) - getG(msgroup_p1)
            pka.pKa_bydG = (dG / C_unit)

            macropkas.append(pka)

        # Compute 0 to -1 transition
        if -1 in formal_charges:
            pka = Macro_pKa()
            pka.molecule = sm_name.split("_")[0]
            pka.transition_from = 0
            pka.transition_to = -1
            pka.SEM = state_details[0][3]
            pka.MU = state_details[0][4]

            # titration method given by David's group
            init_guess = 5
            func_0neg1 = lambda pH: (pop_charge(pH, -1, state_details) - pop_charge(pH, 0, state_details))
            pH_solution_0toneg1, infodict, ier, mesg = fsolve(func_0neg1, init_guess, factor=0.1, full_output=True)
            if 'The iteration is not making good progress' in mesg:
                init_guess+=3
            pH_solution_0toneg1, infodict, ier, mesg = fsolve(func_0neg1, init_guess, factor=0.1, full_output=True)
            # If still poor convergence, print warning (MAY NEED BETTER SOLUTION TO THIS)
            if 'The iteration is not making good progress' in mesg:
                print(sm_name)
                print("WARNING: Numerical problems encountered with fsolv")
            pka.pKa_bytitration = pH_solution_0toneg1

            # delta G method given by Junjun Mao
            dG = getG(msgroup_n1) - getG(msgroup_p0)
            pka.pKa_bydG = (dG / C_unit)

            macropkas.append(pka)

    return macropkas

## Fix Units

In [5]:
def submission_fix_and_convert(submission_data):
    for submission in submission_data:
        sub = submission.data
        #reset the index so that each prediction can be accessed individually (without having to rearrange reference state and microstate)
        submission.data=submission.data.rename_axis('Molecule ID').reset_index()
        for mol_ID, series in submission.data.iterrows():
            pKa_mean_pred = submission.data.loc[mol_ID, "pKa mean"]
            pKa_SEM_pred = submission.data.loc[mol_ID, "pKa SEM"]
            pKa_model_uncertainty =  submission.data.loc[mol_ID, "pKa model uncertainty"]

            # Convert submissions to kcal/mol
            if submission.file_name in ["pKa-ECRISM-1", "pKa-VA-2-charge-correction", "pKa_RodriguezPaluch_SMD_1", "pKa_RodriguezPaluch_SMD_2", "pKa_RodriguezPaluch_SMD_3"]:
                pKa_mean_pred = pKa_mean_pred*C_unit #convert submission to kcal/mol
                pKa_SEM_pred = pKa_SEM_pred*C_unit
                pKa_model_uncertainty = pKa_model_uncertainty*C_unit

            # fix submission which seems to be in kJ/mol
            if submission.file_name in ["pka-nhlbi-1c"]:
                # correct free energies into kcal/mol
                # submission seemed to have used C_units = 5.69 for kJ/mol, so can divide by 4.186 to get kcal/mol
                pKa_mean_pred = pKa_mean_pred/4.186
                pKa_SEM_pred = pKa_SEM_pred/4.186
                pKa_model_uncertainty = pKa_model_uncertainty/4.186

            #If single transition states are opposite in sign from macro pKa, we assume they made a sign error
            if submission.file_name in [ "pKa-VA-2-charge-correction", "pka-nhlbi-1c", "pKa_RodriguezPaluch_SMD_1","pKa_RodriguezPaluch_SMD_2", "pKa_RodriguezPaluch_SMD_3"]:
                pKa_mean_pred = pKa_mean_pred*-1 #fix sign error
                submission.data.loc[mol_ID, "pKa mean"] = pKa_mean_pred

            submission.data.loc[mol_ID, "pKa mean"] = pKa_mean_pred
            submission.data.loc[mol_ID, "pKa SEM"] = pKa_SEM_pred
            submission.data.loc[mol_ID, "pKa model uncertainty"] = pKa_model_uncertainty
        submission.data = submission.data.set_index('Molecule ID')
    return submission_data

In [6]:
microstate_dt_fixed = submission_fix_and_convert(microstate_dt)

In [ ]:
macrostates = microstate_dt_fixed[0].data[0:0]
del macrostates['ID tag']
del macrostates['total charge']
for microstates in microstate_dt_fixed:
    macropKas = get_macropka(microstates.data)
    for pka in macropKas:
        macrostates = macrostates.append({"Molecule ID": str(pka.molecule),
                                          "pKa mean": pka.pKa_bytitration[0],
                                          "pKa SEM": pka.SEM,
                                          "pKa model uncertainty":pka.MU},
                                         ignore_index = True)
        macrostates.set_index('Molecule ID')


In [10]:
microstate_dt_fixed[0].data


,ID tag,total charge,pKa mean,pKa SEM,pKa model uncertainty
Molecule ID,,,,,
SAMPL8-1_micro000,SAMPL8-1_extra001,1,1.10,0.1,1.4
SAMPL8-1_micro000,SAMPL8-1_extra002,1,1.12,0.1,1.4
SAMPL8-1_micro000,SAMPL8-1_micro000,0,0.00,0.1,1.4
SAMPL8-1_micro000,SAMPL8-1_micro007,0,15.42,0.1,1.4
SAMPL8-1_micro000,SAMPL8-1_micro010,0,14.07,0.1,1.4
...,...,...,...,...,...
SAMPL8-9_micro002,SAMPL8-9_micro002,1,-10.60,0.1,1.4
SAMPL8-9_micro002,SAMPL8-9_micro004,1,-0.11,0.1,1.4
SAMPL8-9_micro002,SAMPL8-9_micro000,0,0.00,0.1,1.4
